In [1]:
from elasticsearch import Elasticsearch
import os
import pandas as pd
import sys
import json
from smappdragon import JsonCollection
import tornado.options

In [12]:
# import os
# os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home'
# os.environ['CLASSPATH'] = '/Users/elle0804/spark-2.4.2-bin-hadoop2.7/jars/scala-library-2.12.8.jar'

In [3]:
es = Elasticsearch()
es.info(pretty = True)

{'name': 'TeXop4i',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'u7bKaI3bTQCU9rM0vdpWUg',
 'version': {'number': '5.3.0',
  'build_hash': '3adb13b',
  'build_date': '2017-03-23T03:31:50.652Z',
  'build_snapshot': False,
  'lucene_version': '6.4.1'},
 'tagline': 'You Know, for Search'}

In [4]:
path = os.getcwd()
data_dir = os.path.join(path, '../data')
data_path = os.path.join(path, '../data/TrSample2015.json')
collection = JsonCollection(data_path)

In [5]:
for (i, tweet) in enumerate(collection.get_iterator()):
    print(tweet)
    break

{'id': 621246888029782016, 'text': 'current weather in Portland: cloudy, 66°F\n88% humidity, wind 0mph, visibility 1mi, pressure 29.6in steady', 'created_at': 'Wed Jul 15 09:16:14 +0000 2015', 'user': {'location': 'Portland, ME'}}


In [6]:
spark

In [ ]:
# from pyspark.sql import SparkSession, SQLContext
# from pyspark.sql.types import *
# spark = SparkSession.builder.master("local").appName("Fuzzy Search").getOrCreate()

In [7]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)
doc = spark.read.load(data_path, format="json")
doc.show()

+--------------------+------------------+--------------------+--------------------+
|          created_at|                id|                text|                user|
+--------------------+------------------+--------------------+--------------------+
|Wed Jul 15 09:16:...|621246888029782016|current weather i...|      [Portland, ME]|
|Wed Jul 15 10:16:...|621262104004329474|15 July 2015 #dai...|              [SoPo]|
|Wed Jul 15 12:02:...|621288821792112644|Good morning frie...|                [nh]|
|Wed Jul 15 09:16:...|621247071538941953|Current Condition...|[Brunswick, Maine...|
|Wed Jul 15 06:14:...|621201038947684356|cloudy -&gt; fog
...|      [Portland, ME]|
|Wed Jul 15 11:35:...|621281862275715072|Let's Shopping 🇺...|[Bangkok ✈️ Thail...|
|Wed Jul 15 11:55:...|621286977447546880|#LoveTheSkinYoure...|       [York, Maine]|
|Wed Jul 15 09:12:...|621245881115766784|5:14am: sunrise
s...|      [Portland, ME]|
|Wed Jul 15 12:32:...|621296182799810560|@anniehoffman @so...|[iPhone: 43.671

In [8]:
schema = {
    "mappings":{
        "twitter2015":{
            "properties":{
             "id": {"type": "text"},
             "text": {"type": "text"},
             "created_at": {"type": "text"},
              "user": {"type": "nested",
                        "properties": {
                         "location": {"type": "text"}
                     }
                }
            }
        }     
    } 
}
es.indices.delete(index="demo")
es.indices.create(index="demo", body=schema)

{'acknowledged': True, 'shards_acknowledged': True}

In [9]:
# Type is specified here, which means we can always add another type to the index by this function. 
doc.write.format("es").save("demo/twitter2015")

In [10]:
fuzzy = {
        "query": {
            "fuzzy": {
                "text": {
                    "value": "cloud",
                    "fuzziness": 2,
            }
        }
    }
}

In [11]:
fuzzy_search = es.search(index="demo", body = fuzzy)

In [12]:
fuzzy_search

{'took': 146,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'failed': 0},
 'hits': {'total': 32630,
  'max_score': 7.8177214,
  'hits': [{'_index': 'demo',
    '_type': 'twitter2015',
    '_id': 'AWr2-CacmHCcLGwVem36',
    '_score': 7.8177214,
    '_source': {'created_at': 'Mon Sep 28 15:29:15 +0000 2015',
     'id': 648519847744290816,
     'text': 'Silly cloud #clouds #killntime @ Tanner Park https://t.co/ZfoQ1SRQvM',
     'user': {'location': 'Utah, USA'}}},
   {'_index': 'demo',
    '_type': 'twitter2015',
    '_id': 'AWr2-F4jmHCcLGwVfWPP',
    '_score': 7.2279444,
    '_source': {'created_at': 'Sun Nov 22 18:07:25 +0000 2015',
     'id': 668490984561381376,
     'text': "It's all about the cloth cord!!! In stock in cloud, earth, and sky. Cloth Cord is sold individually… https://t.co/2jeCBw3uN9",
     'user': {'location': 'Tulsa, Oklahoma'}}},
   {'_index': 'demo',
    '_type': 'twitter2015',
    '_id': 'AWr2-EKmmHCcLGwVfADP',
    '_score': 6.568062,
    '_source':

In [18]:
print("Got {} Hits for key word \"cloud\". First 10 id:".format(fuzzy_search['hits']['total']))
for hit in fuzzy_search['hits']['hits']:
    print(hit["_source"]["id"])

Got 32630 Hits for key word "cloud". First 10 id:
648519847744290816
668490984561381376
639948239136030720
643814368577220608
652466067772542976
639949060129091584
615466151347359744
638801184573272064
648838017474527232
606590642286936064
